In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from data_cleaning import Data_Preprocessing
from arabert.preprocess import ArabertPreprocessor

In [ ]:
# Core
import random

# Basics
import numpy as np
import pandas as pd
import torch

# Metrics
from sklearn.metrics import *

# Tokeniser
from transformers import BertTokenizer

# Utility
from tqdm import tqdm

# Dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Scheduler
from transformers import get_linear_schedule_with_warmup

# Optimiser
from transformers import AdamW

# Model
from transformers import BertForSequenceClassification
import torch.nn as nn

class BERT:
    def __init__(self,args):
        # fix the random
        random.seed(args['seed_val'])
        np.random.seed(args['seed_val'])
        torch.manual_seed(args['seed_val'])
        torch.cuda.manual_seed_all(args['seed_val'])
        
        # set device
        self.device = torch.device(args['device'])

        self.weights=args['weights']
        
        # initiliase tokeniser
        self.tokenizer = BertTokenizer.from_pretrained(args['bert_model'])

        self.model_save_path = args['model_save_path']
        self.name = args['name']
        
    ##-----------------------------------------------------------##
    ##----------------- Utility Functions -----------------------##
    ##-----------------------------------------------------------##
    def encode(self,data,max_len):
        input_ids = []
        attention_masks = []
        for sent in tqdm(data):
            # use in-built tokeniser of Bert
            encoded_dict = self.tokenizer.encode_plus(
                            sent,
                            add_special_tokens =True, # for [CLS] and [SEP]
                            max_length = max_len,
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,
                            return_tensors = 'pt', # return pytorch tensors
            )
            input_ids.append(encoded_dict['input_ids'])
            # attention masks notify where padding has been added 
            # and where is the sentence
            attention_masks.append(encoded_dict['attention_mask'])
        
        return [input_ids,attention_masks]
    
    ##-----------------------------------------------------------##
    ##------------------ Dataloader -----------------------------##
    ##-----------------------------------------------------------##
    def get_dataloader(self,samples, batch_size,is_train=False):
        inputs,masks,labels = samples

        # Convert the lists into tensors.
        inputs = torch.cat(inputs, dim=0)
        masks = torch.cat(masks, dim=0)
        labels = torch.tensor(labels)

        # convert to dataset
        data = TensorDataset(inputs,masks,labels)

        if(is_train==False):
            # use random sampler for training to shuffle
            # train data
            sampler = SequentialSampler(data)
        else:
            # order does not matter for validation as we just 
            # need the metrics
            sampler = RandomSampler(data)  

        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size,drop_last=True)

        return dataloader
    
    ##-----------------------------------------------------------##
    ##----------------- Training Utilities ----------------------##
    ##-----------------------------------------------------------## 
    def get_optimiser(self,learning_rate,model):
        # using AdamW optimiser from transformers library
        return AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8
                )
    
    def get_scheduler(self,epochs,optimiser,train_dl):
        total_steps = len(train_dl) * epochs
        return get_linear_schedule_with_warmup(optimiser, 
                num_warmup_steps = 0, 
                num_training_steps = total_steps)
    
    def evalMetric(self, y_true, y_pred, prefix):
        # calculate all the metrics and add prefix to them
        # before saving in dictionary
        accuracy = accuracy_score(y_true, y_pred)
        mf1Score = f1_score(y_true, y_pred, average='macro')
        f1Score = f1_score(y_true, y_pred)
        area_under_c = roc_auc_score(y_true, y_pred)
        recallScore = recall_score(y_true, y_pred)
        precisionScore = precision_score(y_true, y_pred)

        nonhate_f1Score = f1_score(y_true, y_pred, pos_label=0)
        non_recallScore = recall_score(y_true, y_pred, pos_label=0)
        non_precisionScore = precision_score(y_true, y_pred, pos_label=0)
        return {prefix+"accuracy": accuracy, prefix+'mF1Score': mf1Score, 
            prefix+'f1Score': f1Score, prefix+'auc': area_under_c,
            prefix+'precision': precisionScore, 
            prefix+'recall': recallScore, 
            prefix+'non_hatef1Score': nonhate_f1Score, 
            prefix+'non_recallScore': non_recallScore, 
            prefix+'non_precisionScore': non_precisionScore}
    
    ##-----------------------------------------------------------##
    ##---------------- Different Train Loops --------------------##
    ##-----------------------------------------------------------## 
    def evaluate(self,model,loader,which):
        # to evaluate model on test and validation set

        model.eval() # put model in eval mode

        # maintain total loss to save in metrics
        total_eval_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(loader):
            # separate input, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            with torch.no_grad(): # do not construct compute graph
                outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            # output is always a tuple, thus we have to 
            # separate it manually
            loss = outputs[0]
            logits = outputs[1]

            # add the current loss
            # loss.item() extracts loss value as a float
            total_eval_loss += loss.item()

            # calculate true labels and convert it into numpy array
            b_y_true = b_labels.cpu().data.squeeze().numpy()
            
            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

        # calculate metrics
        metrics = self.evalMetric(y_true,y_pred,which+"_")

        # Calculate the average loss over all of the batches.
        avg_loss = total_eval_loss / len(loader)
        # add it to the metric
        metrics[which+'_avg_loss'] = avg_loss

        return metrics
    
    
    def run_train_loop(self,model,train_loader,optimiser,scheduler):

        model.train() # put model in train mode

        # maintain total loss to add to metric
        total_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(train_loader):
            # separate inputs, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            # Ref: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch#:~:text=In%20PyTorch%20%2C%20we%20need%20to,backward()%20call.
            model.zero_grad()                

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

            # outputs is always returned as tuple
            # Separate it manually
            logits = outputs[1]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float)).to(self.device)
            
            loss = loss_fct(logits,b_labels)
            
            # calculate current loss
            # loss.item() extracts loss value as a float
            total_loss += loss.item()

            # Back-propagation
            loss.backward()

            # calculate true labels
            b_y_true = b_labels.cpu().data.squeeze().numpy()

            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

            # clip gradient to prevent exploding gradient
            # problems
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # gradient descent
            optimiser.step()
            
            # schedule learning rate accordingly
            scheduler.step()

        # calculate avg loss 
        avg_train_loss = total_loss / len(train_loader)

        # calculate metrics
        train_metrics = self.evalMetric(y_true,y_pred,"Train_")
        
        # print results
        print('avg_train_loss',avg_train_loss)
        print('train_f1Score',train_metrics['Train_f1Score'])
        print('train_accuracy',train_metrics['Train_accuracy'])

        # add loss to metrics
        train_metrics['Train_avg_loss'] = avg_train_loss

        return train_metrics
    
    
    ##------------------------------------------------------------##
    ##----------------- Main Train Loop --------------------------##
    ##------------------------------------------------------------##
    def train(self,model,data_loaders,optimiser,scheduler,epochs,save_model):
        # save train stats per epoch
        train_stats = []
        train_loader,val_loader,test_loader = data_loaders
        # maintain best mF1 Score to save best model
        best_mf1Score=-1.0
        for epoch_i in range(0, epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            
            print("")
            print('Training...')
            # run trian loop
            train_metrics = self.run_train_loop(model,train_loader,
                                            optimiser,scheduler)

            print("")
            print("Running Validation...") 
            # test on validation set
            val_metrics = self.evaluate(model,val_loader,"Val")
            
            print("Validation Loss: ",val_metrics['Val_avg_loss'])
            print("Validation Accuracy: ",val_metrics['Val_accuracy'])
            
            stats = {}

            # save model where validation mF1Score is best
            if(val_metrics['Val_mF1Score']>best_mf1Score):
                best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate best model on test set
                test_metrics = self.evaluate(model,test_loader,"Test")

            stats['epoch']=epoch_i+1

            # add train and val metrics of the epoch to 
            # same dictionary
            stats.update(train_metrics)
            stats.update(val_metrics)

            train_stats.append(stats)

        return train_stats,test_metrics
    
    ##-----------------------------------------------------------##
    ##----------------------- Main Pipeline ---------------------##
    ##-----------------------------------------------------------##
    def run(self,args,df_train,df_val,df_test):
        # get X and Y data points 
        X_train = df_train['Text'].values
        Y_train = df_train['Label'].values
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values
        X_val = df_val['Text'].values
        Y_val = df_val['Label'].values
        
        # encode data
        # returns list of data and attention masks
        train_data = self.encode(X_train,args['max_len'])
        val_data = self.encode(X_val,args['max_len'])
        test_data = self.encode(X_test,args['max_len'])
        
        # add labels to data so that we can send them to
        # dataloader function together
        train_data.append(Y_train)
        val_data.append(Y_val)
        test_data.append(Y_test)
        
        # convert to dataloader
        train_dl =self.get_dataloader(train_data,args['batch_size'],True)
        val_dl =self.get_dataloader(val_data,args['batch_size'])                          
        test_dl =self.get_dataloader(test_data,args['batch_size'])
        
        # intialise model
        model = BertForSequenceClassification.from_pretrained(
                args['bert_model'], 
                num_labels = 2, 
                output_attentions = False, # Whether the model returns attentions weights.
                output_hidden_states = False, # Whether the model returns all hidden-states.
            )
        model.to(self.device)
        
        optimiser = self.get_optimiser(args['learning_rate'],model)
        
        scheduler = self.get_scheduler(args['epochs'],optimiser,train_dl)
        
        # Run train loop and evaluate on validation data set
        # on each epoch. Store best model from all epochs 
        # (best mF1 Score on Val set) and evaluate it on
        # test set
        train_stats,train_metrics = self.train(model,[train_dl,val_dl,test_dl],
                                optimiser,scheduler,args['epochs'],args['save_model'])
        
        return train_stats,train_metrics
        
    ##-----------------------------------------------------------##
    ##-------------------- Other Utilities ----------------------##
    ##-----------------------------------------------------------##
    def run_test(self,model,df_test,args):
        # to evaluate test set on the final saved model
        # to retrieve results if necessary
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values

        test_data = self.encode(X_test,args['max_len'])

        test_data.append(Y_test)

        test_dl =self.get_dataloader(test_data,32)

        metrics = self.evaluate(model,test_dl,"Test")

        return metrics
    
    def load_model(self,path,args):
        # load saved best model
#         config = BertConfig.from_pretrained(args['bert_model'])
        saved_model = BertForSequenceClassification.from_pretrained(
                args['bert_model'], 
                num_labels = 2, 
                output_attentions = False, # Whether the model returns attentions weights.
                output_hidden_states = False, # Whether the model returns all hidden-states.
            )
#         saved_model.bert.embeddings.word_embeddings=torch.nn.Embedding(64000,768,padding_idx=0)
        
        saved_model.load_state_dict(torch.load(path))
        
        return saved_model

In [ ]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [ ]:
def load_dataset(args,data_path,index):
    # read dataframes
    df_test = pd.read_csv(data_path+'test_'+str(index)+'.csv')

    # clean data
    df_test=preprocess(df_test,args['isArabic'])

    return df_test

In [ ]:
def one_shot_output(model_path,data_path,obj,args):
    saved_model=obj.load_model(model_path,args)
    device = torch.device(args['device'])
    saved_model=saved_model.to(device)
    
    all_metrics=[]
    avg_metrics={}
    
    # preprocessing
    for fold in [1,2,3,4,5]:
        df = load_dataset(args,data_path,fold)

        metrics = obj.run_test(saved_model,df,args)
        
        for key,value in metrics.items():
            if(key not in avg_metrics):
                avg_metrics[key]=value
            else:
                avg_metrics[key]+=value
        
        all_metrics.append(metrics)
            
    
    for key,value in avg_metrics.items():
        avg_metrics[key]/=5
    
    return all_metrics,avg_metrics

In [ ]:
DATA_PATH = "Data_Processed/Let-Mi/all.csv"
MODEL_PATH = "Saved_Models/Let-Mi/best_bert_xlm_roberta_1_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': True,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

In [ ]:
metrics

In [ ]:
# base_model_path = "../../HULK/HateModels/Shrinivas/Misogyny-Analysis/"
base_model_path=""

In [ ]:
def getOneShotOutput(model_path,data_path):
    args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }
    
    if('Let-Mi' in data_path):
        args['isArabic']=True

    model = BERT(args)

    all_metrics, metrics = one_shot_output(base_model_path+model_path,data_path,model,args)
    
    return metrics,all_metrics

In [ ]:
datasets=['Let-Mi','AMI-2020','AMI-Spanish','Shared_Task_eng',
          'Shared_Task_hin','Shared_Task_iben']

In [ ]:
paths={
    'Let-Mi':{
        'model_path':'Saved_Models/Let-Mi/best_bert_1_all.pt',
        'data_path':'Data_Processed/Let-Mi/',
    },
    'AMI-2020':{
        'model_path': 'Saved_Models/AMI-2020/best_bert_1_all.pt',
        'data_path': 'Data_Processed/AMI-2020/',
    },
    'AMI-Spanish':{
        'model_path': 'Saved_Models/AMI-Spanish/best_bert_1_all.pt',
        'data_path': 'Data_Processed/AMI-Spanish/',
    },
    'Shared_Task_eng':{
        'model_path': 'Saved_Models/Shared_Task_eng/best_bert_1_allnew.pt',
        'data_path': 'Data_Processed/Shared_Task_eng/',
    },
    'Shared_Task_iben':{
        'model_path': 'Saved_Models/Shared_Task_iben/best_bert_4_all.pt',
        'data_path': 'Data_Processed/Shared_Task_iben/',
    },
    'Shared_Task_hin':{
        'model_path': 'Saved_Models/Shared_Task_hin/best_bert_5_all.pt',
        'data_path': 'Data_Processed/Shared_Task_hin/',
    },
}

In [ ]:
res = []

In [ ]:
for dataset1 in datasets:
    for dataset2 in datasets:
        if(dataset1!=dataset2):
            name = dataset1+'_'+dataset2
            metrics = getOneShotOutput(paths[dataset1]['model_path'],
                                      paths[dataset2]['data_path'])
            metrics['Name']=dataset1+'_'+dataset2
            res.append(metrics)

In [ ]:
metrics,all_metrics = getOneShotOutput(paths['Shared_Task_iben']['model_path'],
                                      paths['AMI-2020']['data_path'])

In [ ]:
metrics

## XLM Roberta

In [ ]:
from data_cleaning import Data_Preprocessing
from arabert.preprocess import ArabertPreprocessor
import pandas as pd
import numpy as np
import torch
import random
from tqdm import tqdm
from transformers import RobertaPreTrainedModel,RobertaModel

In [ ]:
!nvidia-smi

In [ ]:
# Core
import random

# Basics
import numpy as np
import pandas as pd
import torch

# Metrics
from sklearn.metrics import *

# Tokeniser
from transformers import XLMRobertaTokenizer

# Utility
from tqdm import tqdm

# Dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Scheduler
from transformers import get_linear_schedule_with_warmup

# Optimiser
from transformers import AdamW

# Model

import torch.nn as nn
from models import weighted_Roberta


class XLM_Roberta:
    def __init__(self,args):
        # fix the random
        random.seed(args['seed_val'])
        np.random.seed(args['seed_val'])
        torch.manual_seed(args['seed_val'])
        torch.cuda.manual_seed_all(args['seed_val'])
        
        # set device
        self.device = torch.device(args['device'])

        self.weights=args['weights']
        
        # initiliase tokeniser
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', do_lower_case = True)

        self.model_save_path = args['model_save_path']
        self.name = args['name']
        
    ##-----------------------------------------------------------##
    ##----------------- Utility Functions -----------------------##
    ##-----------------------------------------------------------##
    def encode(self,data,max_len):
        input_ids = []
        attention_masks = []
        for sent in tqdm(data):
            # use in-built tokeniser of Bert
            encoded_dict = self.tokenizer.encode_plus(
                            sent,
                            add_special_tokens =True, # for [CLS] and [SEP]
                            max_length = max_len,
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,
#                             return_tensors = 'pt', # return pytorch tensors
            )
            input_ids.append(encoded_dict['input_ids'])
            # attention masks notify where padding has been added 
            # and where is the sentence
            attention_masks.append(encoded_dict['attention_mask'])
            X_data = torch.tensor(input_ids)
            attention_masks_data = torch.tensor(attention_masks)
            
        return [X_data,attention_masks_data]
    
    ##-----------------------------------------------------------##
    ##------------------ Dataloader -----------------------------##
    ##-----------------------------------------------------------##
    def get_dataloader(self,samples, batch_size,is_train=False):
        inputs,masks,labels = samples

        # Convert the lists into tensors.
#         inputs = torch.cat(inputs, dim=0)
#         masks = torch.cat(masks, dim=0)
        labels = torch.tensor(labels)

        # convert to dataset
        data = TensorDataset(inputs,masks,labels)

        if(is_train==False):
            # use random sampler for training to shuffle
            # train data
            sampler = SequentialSampler(data)
        else:
            # order does not matter for validation as we just 
            # need the metrics
            sampler = RandomSampler(data)  

        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size,drop_last=True)

        return dataloader
    
    ##-----------------------------------------------------------##
    ##----------------- Training Utilities ----------------------##
    ##-----------------------------------------------------------## 
    def get_optimiser(self,learning_rate,model):
        # using AdamW optimiser from transformers library
        return AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8
                )
    
    def get_scheduler(self,epochs,optimiser,train_dl):
        total_steps = len(train_dl) * epochs
        return get_linear_schedule_with_warmup(optimiser, 
                num_warmup_steps = 0, 
                num_training_steps = total_steps)
    
    def evalMetric(self, y_true, y_pred, prefix):
        # calculate all the metrics and add prefix to them
        # before saving in dictionary
        accuracy = accuracy_score(y_true, y_pred)
        mf1Score = f1_score(y_true, y_pred, average='macro')
        f1Score = f1_score(y_true, y_pred)
        area_under_c = roc_auc_score(y_true, y_pred)
        recallScore = recall_score(y_true, y_pred)
        precisionScore = precision_score(y_true, y_pred)

        nonhate_f1Score = f1_score(y_true, y_pred, pos_label=0)
        non_recallScore = recall_score(y_true, y_pred, pos_label=0)
        non_precisionScore = precision_score(y_true, y_pred, pos_label=0)
        return {prefix+"accuracy": accuracy, prefix+'mF1Score': mf1Score, 
            prefix+'f1Score': f1Score, prefix+'auc': area_under_c,
            prefix+'precision': precisionScore, 
            prefix+'recall': recallScore, 
            prefix+'non_hatef1Score': nonhate_f1Score, 
            prefix+'non_recallScore': non_recallScore, 
            prefix+'non_precisionScore': non_precisionScore}
    
    ##-----------------------------------------------------------##
    ##---------------- Different Train Loops --------------------##
    ##-----------------------------------------------------------## 
    def evaluate(self,model,loader,which):
        # to evaluate model on test and validation set

        model.eval() # put model in eval mode

        # maintain total loss to save in metrics
        total_eval_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(loader):
            # separate input, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            with torch.no_grad(): # do not construct compute graph
                outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            # output is always a tuple, thus we have to 
            # separate it manually
            #loss = outputs[0]
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))

            # add the current loss
            # loss.item() extracts loss value as a float
            total_eval_loss += loss.item()

            # calculate true labels and convert it into numpy array
            b_y_true = b_labels.cpu().data.squeeze().numpy()
            
            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

        # calculate metrics
        metrics = self.evalMetric(y_true,y_pred,which+"_")

        # Calculate the average loss over all of the batches.
        avg_loss = total_eval_loss / len(loader)
        # add it to the metric
        metrics[which+'_avg_loss'] = avg_loss

        return metrics
    
    
    def run_train_loop(self,model,train_loader,optimiser,scheduler):

        model.train() # put model in train mode

        # maintain total loss to add to metric
        total_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(train_loader):
            # separate inputs, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            # Ref: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch#:~:text=In%20PyTorch%20%2C%20we%20need%20to,backward()%20call.
            model.zero_grad()                

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

            # outputs is always returned as tuple
            # Separate it manually
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))
            
            # calculate current loss
            # loss.item() extracts loss value as a float
            total_loss += loss.item()

            # Back-propagation
            loss.backward()

            # calculate true labels
            b_y_true = b_labels.cpu().data.squeeze().numpy()

            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

            # clip gradient to prevent exploding gradient
            # problems
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # gradient descent
            optimiser.step()
            
            # schedule learning rate accordingly
            scheduler.step()

        # calculate avg loss 
        avg_train_loss = total_loss / len(train_loader)

        # calculate metrics
        train_metrics = self.evalMetric(y_true,y_pred,"Train_")
        
        # print results
        print('avg_train_loss',avg_train_loss)
        print('train_f1Score',train_metrics['Train_f1Score'])
        print('train_accuracy',train_metrics['Train_accuracy'])

        # add loss to metrics
        train_metrics['Train_avg_loss'] = avg_train_loss

        return train_metrics
    
    
    ##------------------------------------------------------------##
    ##----------------- Main Train Loop --------------------------##
    ##------------------------------------------------------------##
    def train(self,model,data_loaders,optimiser,scheduler,epochs,save_model):
        # save train stats per epoch
        train_stats = []
        train_loader,val_loader,test_loader = data_loaders
        # maintain best mF1 Score to save best model
        best_mf1Score=-1.0
        for epoch_i in range(0, epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            
            print("")
            print('Training...')
            # run trian loop
            train_metrics = self.run_train_loop(model,train_loader,
                                            optimiser,scheduler)

            print("")
            print("Running Validation...") 
            # test on validation set
            val_metrics = self.evaluate(model,val_loader,"Val")
            
            print("Validation Loss: ",val_metrics['Val_avg_loss'])
            print("Validation Accuracy: ",val_metrics['Val_accuracy'])
            
            stats = {}

            # save model where validation mF1Score is best
            if(val_metrics['Val_mF1Score']>best_mf1Score):
                best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate best model on test set
                test_metrics = self.evaluate(model,test_loader,"Test")

            stats['epoch']=epoch_i+1

            # add train and val metrics of the epoch to 
            # same dictionary
            stats.update(train_metrics)
            stats.update(val_metrics)

            train_stats.append(stats)

        return train_stats,test_metrics
    
    ##-----------------------------------------------------------##
    ##----------------------- Main Pipeline ---------------------##
    ##-----------------------------------------------------------##
    def run(self,args,df_train,df_val,df_test):
        # get X and Y data points 
        X_train = df_train['Text'].values
        Y_train = df_train['Label'].values
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values
        X_val = df_val['Text'].values
        Y_val = df_val['Label'].values
        
        # encode data
        # returns list of data and attention masks
        train_data = self.encode(X_train,args['max_len'])
        val_data = self.encode(X_val,args['max_len'])
        test_data = self.encode(X_test,args['max_len'])
        
        # add labels to data so that we can send them to
        # dataloader function together
        train_data.append(Y_train)
        val_data.append(Y_val)
        test_data.append(Y_test)
        
        # convert to dataloader
        train_dl =self.get_dataloader(train_data,args['batch_size'],True)
        val_dl =self.get_dataloader(val_data,args['batch_size'])                          
        test_dl =self.get_dataloader(test_data,args['batch_size'])
        
        # intialise model
        model = weighted_Roberta.from_pretrained(
            'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            params=args['params'],
        )
        model.to(self.device)
        
        optimiser = self.get_optimiser(args['learning_rate'],model)
        
        scheduler = self.get_scheduler(args['epochs'],optimiser,train_dl)
        
        # Run train loop and evaluate on validation data set
        # on each epoch. Store best model from all epochs 
        # (best mF1 Score on Val set) and evaluate it on
        # test set
        train_stats,train_metrics = self.train(model,[train_dl,val_dl,test_dl],
                                optimiser,scheduler,args['epochs'],args['save_model'])
        
        return train_stats,train_metrics
        
    ##-----------------------------------------------------------##
    ##-------------------- Other Utilities ----------------------##
    ##-----------------------------------------------------------##
    def run_test(self,model,df_test,args):
        # to evaluate test set on the final saved model
        # to retrieve results if necessary
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values

        test_data = self.encode(X_test,args['max_len'])

        test_data.append(Y_test)

        test_dl =self.get_dataloader(test_data,32)

        metrics = self.evaluate(model,test_dl,"Test")

        return metrics
    
    def load_model(self,path,args):
        # load saved best model
        saved_model = weighted_Roberta.from_pretrained(
            'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            params=args['params'],
        )
        
        saved_model.load_state_dict(torch.load(path))
        
        return saved_model

In [ ]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [ ]:
def load_dataset(args,data_path,index):
    # read dataframes
    df_test = pd.read_csv(data_path+'test_'+str(index)+'.csv')

    # clean data
    df_test=preprocess(df_test,args['isArabic'])

    return df_test

In [ ]:
def one_shot_output(model_path,data_path,obj,args):
    saved_model=obj.load_model(model_path,args)
    device = torch.device(args['device'])
    saved_model=saved_model.to(device)
    
    all_metrics=[]
    avg_metrics={}
    
    # preprocessing
    for fold in [1,2,3,4,5]:
        df = load_dataset(args,data_path,fold)

        metrics = obj.run_test(saved_model,df,args)
        
        for key,value in metrics.items():
            if(key not in avg_metrics):
                avg_metrics[key]=value
            else:
                avg_metrics[key]+=value
        
        all_metrics.append(metrics)
    
    for key,value in avg_metrics.items():
        avg_metrics[key]/=5
    
    return avg_metrics,all_metrics

In [ ]:
def getOneShotOutput(model_path,data_path):
    model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }
    
    if('Let-Mi' in data_path):
        model_args['isArabic']=True

    model = XLM_Roberta(model_args)

    avg_metrics,all_metrics = one_shot_output(model_path,data_path,model,model_args)
    
    return avg_metrics,all_metrics

In [ ]:
datasets=['Let-Mi','AMI-2020','AMI-Spanish','Shared_Task_eng',
          'Shared_Task_hin','Shared_Task_iben']

In [ ]:
paths={
    'Let-Mi':{
        'model_path':'Saved_Models/Let-Mi/best_bert_xlm_roberta_3_all.pt',
        'data_path':'Data_Processed/Let-Mi/',
    },
    'AMI-2020':{
        'model_path': 'Saved_Models/AMI-2020/best_bert_xlm_roberta_2_all.pt',
        'data_path': 'Data_Processed/AMI-2020/',
    },
    'AMI-Spanish':{
        'model_path': 'Saved_Models/AMI-Spanish/best_bert_xlm_roberta_1_all.pt',
        'data_path': 'Data_Processed/AMI-Spanish/',
    },
    'Shared_Task_eng':{
        'model_path': 'Saved_Models/Shared_Task_eng/best_bert_xlm_roberta_4_all.pt',
        'data_path': 'Data_Processed/Shared_Task_eng/',
    },
    'Shared_Task_iben':{
        'model_path': 'Saved_Models/Shared_Task_iben/best_bert_xlm_roberta_4_all.pt',
        'data_path': 'Data_Processed/Shared_Task_iben/',
    },
    'Shared_Task_hin':{
        'model_path': 'Saved_Models/Shared_Task_hin/best_bert_xlm_roberta_1_all.pt',
        'data_path': 'Data_Processed/Shared_Task_hin/',
    },
}

In [ ]:
res=[]

In [ ]:
for dataset1 in datasets:
    for dataset2 in datasets:
        if(dataset1!=dataset2):
            name = dataset1+'_'+dataset2
            avg_metrics,all_metrics = getOneShotOutput(paths[dataset1]['model_path'],
                                      paths[dataset2]['data_path'])
            metrics['Name']=dataset1+'_'+dataset2
            res.append(metrics)
            df = pd.DataFrame(res)
            df.to_csv('Results_Processed/one_shot_matrix_xlm.csv')

In [ ]:
avg_metrics,all_metrics = getOneShotOutput(paths['Shared_Task_iben']['model_path'],
                                      paths['Shared_Task_hin']['data_path'])

In [ ]:
all_metrics